# Pipeline of Pando

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R 
options(digits=5, max.print=100)  # Adjust numbers as needed
set.seed(123)

# library(Pando)
library(Seurat)
library(BSgenome.Hsapiens.UCSC.hg38)

# # Get motif data
# data(motifs)

# packageVersion('Seurat')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp
‘SeuratObject’ was built under R 4.1.2 but the current version is
4.3.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed
‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed

Attaching package: ‘SeuratObject’

The following object is masked from ‘package:base’:

    intersect

Loading required package: GenomeInfoDb
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following object is masked from ‘package:SeuratObject’:

    intersect

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, i

In [5]:
%%R
library(Matrix)

X <- readMM("../output/scATAC/matrix_annot/X_matrix.mtx")
X <- t(X)
annotation_peak <- read.csv("../output/scATAC/matrix_annot/annotation_peak.csv", row.names = 1)
annotation_cells <- read.csv("../output/scATAC/matrix_annot/annotation_cells.csv", row.names = 1)

# Filter out entries where seqname is not chr 
filter_indices <- grepl("^chr", annotation_peak$seqname)
annotation_peak_filtered <- annotation_peak[filter_indices, ]

# Filter the rows in X
X_filtered <- X[filter_indices, ]
  
peaks_matrix <- X_filtered
colnames(peaks_matrix) <- annotation_cells$obs_id 

rownames(peaks_matrix) = paste(annotation_peak_filtered$seqname, annotation_peak_filtered$ranges, sep = "_")



Attaching package: ‘Matrix’

The following object is masked from ‘package:S4Vectors’:

    expand



In [8]:
%%R 
library(Signac)
library(EnsDb.Hsapiens.v86)
atac_assay <- CreateChromatinAssay(counts = peaks_matrix, ranges=GRanges(annotation_peak_filtered$seqname,
                              IRanges(annotation_peak_filtered$ranges)), 
                             colData = DataFrame(annotation_cells), genome = "hg38")
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "hg38"
Annotation(atac_assay) <- annotations


  |                                                  | 0 % ~calculating   |++                                                | 4 % ~48s           |++++                                              | 8 % ~34s           |++++++                                            | 12% ~38s           |++++++++                                          | 16% ~38s           |++++++++++                                        | 20% ~34s           |++++++++++++                                      | 24% ~30s           |++++++++++++++                                    | 28% ~28s           |++++++++++++++++                                  | 32% ~26s           |++++++++++++++++++                                | 36% ~23s           |++++++++++++++++++++                              | 40% ~21s           |++++++++++++++++++++++                            | 44% ~20s           |++++++++++++++++++++++++                          | 48% ~18s           |++++++++++++++++++++++++++                        | 52% ~16s 

Loading required package: ensembldb
Loading required package: GenomicFeatures
Loading required package: AnnotationDbi
Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.

Loading required package: AnnotationFilter

Attaching package: 'ensembldb'

The following object is masked from 'package:stats':

    filter

In addition: There were 24 warnings (use warnings() to see them)


In [10]:
%%R 
rna_short  = readRDS('../output/FigR/rna_short.rds')
seurat_object <- CreateSeuratObject(count = rna_short, project = "pbmc", min.cells = 3, min.features = 200)
seurat_object[['peaks']] = atac_assay

In [11]:
%%R 
# Assuming your Seurat object is named 'seurat_object'
# seurat_object <- initiate_grn(seurat_object)
seurat_object

An object of class Seurat 
157452 features across 7830 samples within 2 assays 
Active assay: RNA (22272 features, 0 variable features)
 1 layer present: counts
 1 other assay present: peaks


In [1]:
options(digits=5, max.print=100)  # Adjust numbers as needed
set.seed(123)

# library(Pando)
library(Seurat)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Signac)
library(EnsDb.Hsapiens.v86)

library(Matrix)
#-------- import atac-seq count matrix and metadata and creat assay
X <- readMM("X_matrix.mtx")
X <- t(X)
annotation_peak <- read.csv("annotation_peak.csv", row.names = 1)
annotation_cells <- read.csv("annotation_cells.csv", row.names = 1)

# Filter out entries where seqname is not chr 
filter_indices <- grepl("^chr", annotation_peak$seqname)
annotation_peak_filtered <- annotation_peak[filter_indices, ]

# Filter the rows in X
X_filtered <- X[filter_indices, ]
  
peaks_matrix <- X_filtered
colnames(peaks_matrix) <- annotation_cells$obs_id 

rownames(peaks_matrix) = paste(annotation_peak_filtered$seqname, annotation_peak_filtered$ranges, sep = "_")
atac_assay <- CreateChromatinAssay(counts = peaks_matrix, ranges=GRanges(annotation_peak_filtered$seqname,
                              IRanges(annotation_peak_filtered$ranges)), 
                             colData = DataFrame(annotation_cells), genome = "hg38")
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "hg38"
Annotation(atac_assay) <- annotations
#-------- import rna-seq count matrix (with row and col names already assigned ) and create seurat object
rna_short  = readRDS('rna_short.rds')
seurat_object <- CreateSeuratObject(count = rna_short, project = "pbmc", min.cells = 3, min.features = 200)
# add peaks
seurat_object[["peaks"]] = atac_assay

seurat_object

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.1.2 but the current version is
4.3.2; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: GenomeInfoDb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, gre

An object of class Seurat 
157452 features across 7830 samples within 2 assays 
Active assay: RNA (22272 features, 0 variable features)
 1 layer present: counts
 1 other assay present: peaks